# Congress.gov Nomination Details Verification
This notebook helps verify the processing of nomination details from the Congress.gov API.

In [ ]:
import os
import sys
from pathlib import Path

import pandas as pd
from loguru import logger

# Add project root to path
project_root = Path().absolute().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from nomination_predictor.congress_api import (CongressAPIClient,
                                               transform_nomination_data)

# Configure logging
logger.remove()  # Remove default handler
logger.add(
    sys.stderr,
    format="<green>{time:YYYY-MM-DD HH:mm:ss}</green> | <level>{level: <8}</level> | <cyan>{name}</cyan>:<cyan>{function}</cyan>:<cyan>{line}</cyan> - <level>{message}</level>"
)

2025-07-11 04:39:55.912 | INFO     | nomination_predictor.config:<module>:103 - Project root: /home/wsl2ubuntuuser/nomination_predictor
2025-07-11 04:39:55.913 | INFO     | nomination_predictor.config:<module>:127 - Configuration loaded


5

In [ ]:
def get_and_display_nomination(congress: int, nomination_number: int, part_number: str = None):
    """
    Fetch and display nomination details in a user-friendly format.
    
    Args:
        congress: Congress number (e.g., 118)
        nomination_number: Nomination number
        part_number: Optional part number for partitioned nominations
    """
    try:
        client = CongressAPIClient()
        
        # 1. Get the nomination details
        logger.info(f"Fetching details for nomination {nomination_number} (Congress {congress})")
        detail = client.get_nomination_detail(congress, nomination_number, part_number)
        
        # 2. Transform the data
        logger.info("Transforming nomination data...")
        records = transform_nomination_data(detail, full_details=True)
        
        # 3. Display the raw and transformed data
        print(
            "\n===== NOMINATION DETAILS =====\n"
            f"Congress: {congress}\n"
            f"Nomination #: {nomination_number}\n"
            f"Description: {detail.get('nomination', {}).get('description', 'N/A')}\n"
            f"Received Date: {detail.get('nomination', {}).get('receivedDate', 'N/A')}\n"
        )
        
        # Display nominees
        if 'nominees' in detail and detail['nominees']:
            print("\nNOMINEES:")
            for i, nominee in enumerate(detail['nominees'], 1):
                print(
                    f"  {i}. {nominee.get('fullName', 'Unknown')}\n"
                    f"     Position: {nominee.get('positionTitle', 'N/A')}\n"
                    f"     Organization: {nominee.get('organization', 'N/A')}\n"
                    f"     State: {nominee.get('state', 'N/A')}"
                )
                
        # Display actions if available
        if 'actions' in detail and detail['actions']:
            print("\nACTIONS:")
            for action in detail['actions'][:5]:  # Show first 5 actions
                print(
                    f"  {action.get('actionDate', 'N/A')}: {action.get('text', 'N/A')}"
                )
            if len(detail['actions']) > 5:
                print(f"  ... and {len(detail['actions']) - 5} more actions")
        
        # 4. Display the transformed records as a DataFrame
        if records:
            print("\n\n===== TRANSFORMED RECORDS =====\n")
            df = pd.DataFrame(records)
            # Reorder columns for better readability
            columns = [
                'nominee', 'position', 'organization', 'court', 'circuit', 'state',
                'nomination_number', 'congress', 'nomination_date', 'confirmation_date',
                'status', 'url'
            ]
            # Only include columns that exist in the DataFrame
            columns = [col for col in columns if col in df.columns]
            display(df[columns])
            return df
        else:
            print("No records were generated from the transformation.")
            return None
            
    except Exception as e:
        logger.error(f"Error processing nomination: {str(e)}")
        if 'detail' in locals() and detail:
            logger.debug("Raw API response:")
            logger.debug(detail)
        raise

## Example Usage

Run the cell below with different congress and nomination numbers to test the API.

Example values to try:
- Congress: 118, Nomination: PN1
- Congress: 117, Nomination: PN100
- Congress: 116, Nomination: PN50

In [ ]:
# Example: Get details for a specific nomination
result_df = get_and_display_nomination(
    congress=118,  # Congress number
    nomination_number=1,  # Nomination number (PN1, PN2, etc.)
    # part_number=1  # Uncomment and set if needed for partitioned nominations
)

# The result_df contains the transformed records as a pandas DataFrame
# You can access it for further analysis if needed

2025-07-11 04:39:56 | INFO     | __main__:get_and_display_nomination:14 - Fetching details for nomination 1 (Congress 118)
2025-07-11 04:39:56 | INFO     | nomination_predictor.congress_api:get_nomination_detail:298 - Fetching detail for nomination 1 (118th Congress)
2025-07-11 04:39:56 | DEBUG    | nomination_predictor.congress_api:get_nomination_detail:305 - Received detail for nomination 1
2025-07-11 04:39:56 | DEBUG    | nomination_predictor.congress_api:get_nomination_detail:310 - Nomination description: Brendan Owens, of Virginia, to be an Assistant Secretary of Defense, vice Lucian Niemeyer.
2025-07-11 04:39:56 | INFO     | __main__:get_and_display_nomination:18 - Transforming nomination data...
2025-07-11 04:39:56 | DEBUG    | nomination_predictor.congress_api:transform_nomination_data:166 - Transforming nomination data (full_details=True)



===== NOMINATION DETAILS =====
Congress: 118
Nomination #: 1
Description: Brendan Owens, of Virginia, to be an Assistant Secretary of Defense, vice Lucian Niemeyer.
Received Date: 2023-01-03

No records were generated from the transformation.
